| ![EEW logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/eew.jpg?raw=true) | ![EDGI logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/edgi.png?raw=true) |
|---|---|

#### This notebook is licensed under GPL 3.0. Please visit our Github repo for more information: 
#### The notebook was collaboratively authored by the Environmental Data & Governance Initiative (EDGI) following our authorship protocol: https://docs.google.com/document/d/1CtDN5ZZ4Zv70fHiBTmWkDJ9mswEipX6eCYrwicP66Xw/
#### For more information about this project, visit https://www.environmentalenforcementwatch.org/

## How to Run this Notebook
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue. 
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

---

# Watershed statistics from ECHO

Here we load some helper code to get us going. If your environment already has these loaded this cell may be skipped. (If you're not sure, it's best to run this cell!)

In [ ]:
# We have a folder of chunks of reusable code that we're using across different
#  Notebooks. This step goes and gets the relevant code from that folder so we
#  can use it here. (https://github.com/edgi-govdata-archiving/ECHO_modules/tree/watershed-geo)
!git clone https://github.com/edgi-govdata-archiving/ECHO_modules.git -b watershed-geo 2> clone.txt 1> stderr.txt

# Geopandas is an open source library for working with geographic data using the
#   data structures library "pandas" (common in Python for data processing).
#   (https://geopandas.org/)
!pip install geopandas 1>> stderr.txt

# Topojson is an open source library that lets us keep file sizes small when
#   working with geographic data, so the Notebooks can run faster while still
#   working with detailed shapes. (https://github.com/mattijn/topojson)
!pip install topojson 1>> stderr.txt
# This code block will print a lot of data as it fetches and installs the libraries
#   Specified above. When it's done, the line below lets us know by printing "Done!"
print("Done!")

This cell must be run to bring in some utility functions.

In [ ]:
# The previous step installed the libraries (explained above); this tells our
#   Notebook how to refer to them.
import geopandas as geopandas
import topojson as tp

# These code blocks come from our folder (https://github.com/edgi-govdata-archiving/ECHO_modules/blob/watershed-geo/make_data_sets.py)
# Each of the files contains a series of function definitions. By running
#   those files here, we make the functions available in this Notebook.
%run ECHO_modules/DataSet.py
%run ECHO_modules/utilities.py
%run ECHO_modules/make_data_sets.py
print("Done!")

Select the state you want to look more closely at. 

In [ ]:
# Create a widget that can be used to select a state
states_w = show_state_widget()

Run this cell to load watershed data. It may take some time depending on the state you're looking at!  (Watershed boundaries are complex polygons, and if the state has many of them it is a lot of data to load.)

In [ ]:
# In this cell, we are building a query using the SQL language so we can get
#   information from the database about the places we're interested in

# Use the selections from the state-selecting widget from the previous block
#   to create a list of the states we're interested in
states_where = ' WHERE states LIKE \'%' + states_w.value + '%\''

# Load data from the Stony Brook University mirror of EPA's ECHO database
# https://gis.stackexchange.com/questions/112057/sql-query-to-have-a-complete-geojson-feature-from-postgis

# This builds the query by appending details about our request to the variable "sql"
sql = """
    SELECT jsonb_build_object(
        'type', 'FeatureCollection', 'features', jsonb_agg(features.feature)
    )
    FROM (
        SELECT jsonb_build_object(
            'type', 'Feature','id', gid, 'geometry',
            ST_AsGeoJSON(geom)::jsonb,'properties',
            to_jsonb(inputs) - 'gid' - 'geom'
        ) AS feature
        FROM (
            SELECT *
            FROM "wbdhu8"
"""
sql += states_where
sql += """
        ) inputs
    ) features;
"""

# The URL here is the location of the database we're requesting from
url= 'http://portal.gss.stonybrook.edu/echoepa/index2.php?query=' # Old server: 'http://apps.tlt.stonybrook.edu/echoepa/index2.php?query='
data_location=url+urllib.parse.quote_plus(sql) + '&pg'
print(sql) # For debugging
print(data_location) # For debugging

# Here, we check to make sure we can read the data we got back
gdf=None
try:
  gdf = geopandas.read_file(data_location)
  print("Data loaded. Now to map it!")
except:
  print('Something went wrong!')

Map the watersheds in this state. It may take some time depending on the state you're looking at! It may also "disconnect" if there are a lot of watersheds. Still, you can just skip to the next step ("Pick a specific watershed...")

In [ ]:
# Load map data
import json

x = tp.Topology(gdf, toposimplify=.01) # Simplify and topologize the watershed boundaries in order to reduce the file size
x = x.to_json() # save as topojson
y = json.loads(x) # load as json

# create the map using a library called Folium (https://github.com/python-visualization/folium)
m = folium.Map()
w = folium.TopoJson(
    y,
    'objects.data',
    name = "Watersheds",
).add_to(m)
folium.GeoJsonTooltip(fields=["name"]).add_to(w)

# compute boundaries so that the map automatically zooms in
b1 = [list(gdf.total_bounds)[1],list(gdf.total_bounds)[0]]
b2 = [list(gdf.total_bounds)[3],list(gdf.total_bounds)[2]]
bounds = [b1,b2] #m.fit_bounds([[52.193636, -2.221575], [52.636878, -1.139759]])
m.fit_bounds(bounds)

# display the map!
display(m)

Pick specific watersheds. Ctrl+click to add to the selection, Shift+click to extend the selection.

In [ ]:
try :
    gdf.set_index("name", inplace=True) # Edit to not alter the gdf
except KeyError:
    # If the cell is run twice the index is already set and "name" is not available
    pass

# Find the "HUC" codes, which are a proxy the EPA uses for watersheds, in the regions of interest  
hucs = gdf.index.unique()

# Set up a widget for selecting watersheds of interest
watershed_w = widgets.SelectMultiple(
    options= hucs.sort_values(),
    description='Watershed:',
    disabled=False,
)

# Display the widget
watershed_w

Get basic information about facilities in this watershed. If no facilities are found, see the "Alternative" section below.

In [ ]:
# Create a list of all HUCs selected by ID 
hucs = []
for x in watershed_w.value:
    huc = gdf.at[ x, "huc8" ]
    huc = float( huc )
    hucs.append( huc )
# Fetch the relevant data from the database 
full_echo_data = get_active_facilities( state=None, region_type='Watershed',
                                      regions_selected=hucs )
use_zips = False
if ( not full_echo_data.empty ):
    display( full_echo_data )

### Alternative, if facilities are not found

If the previous query returned facilities successfully you can skip the next few cells, down to "Map these facilities".

The EPA's data is not always consistent with its watershed identifiers.  If no facilities are returned in the previous cell, you can identify facilities instead by zip codes that are in the watershed region.

Here is a map for looking up zip codes: https://www.unitedstateszipcodes.org/.  Here is another: https://zipmap.net/.

You can enter multiple zip codes separated by commas.  

In [ ]:
use_zips = True
zip_widget = None
zip_widget = show_pick_region_widget( type='Zip Code' )

In [ ]:
zips_selected = get_regions_selected( region_type='Zip Code', region_widget=zip_widget )
full_echo_data = get_active_facilities( state=None, region_type='Zip Code', 
                                  regions_selected=zips_selected )
display( full_echo_data )

### Map these facilities.

In [ ]:
# Create a map of the facilities in the specified watershed area(s) using Folium
if ( full_echo_data is not None ):
    map_of_facilities = mapper(full_echo_data) # Some errors not caught here...
    this_watershed = gdf.loc[gdf.index.isin( watershed_w.value )]
    w = folium.GeoJson(
      this_watershed,
      name = "Watershed",
    ).add_to(map_of_facilities)
    display( map_of_facilities ) 
else:
    print( "There are no facilities in the watershed." ) 

Are the facilities complying with the Clean Water Act? Top violators over the past 13 quarters.

In [ ]:
# Use the function get_top_violators from our imported utilities.py file (https://github.com/edgi-govdata-archiving/ECHO_modules/blob/watershed-geo/utilities.py)
df_violators = get_top_violators( full_echo_data, 'NPDES_FLAG', 
        'CWA_13QTRS_COMPL_HISTORY', 'CWA_FORMAL_ACTION_COUNT', 90 )
display( chart_top_violators( df_violators, watershed_w.value, "Watershed", 'CWA' ))

Get more detailed, program-specific data for these facilities (e.g. longer-term historical [non]compliance with the Clean Water Act). First, select the program.

In [ ]:
# Define the data sets we're interested in learning about
data_set_list = ['CWA Violations', 'CWA Inspections', 'CWA Penalties', 
                 '2020 Discharge Monitoring',] 

# Use the function make_data_sets (imported from https://github.com/edgi-govdata-archiving/ECHO_modules/blob/watershed-geo/make_data_sets.py)
data_sets = make_data_sets( data_set_list )

# Use the function show_data_set_widget (imported from https://github.com/edgi-govdata-archiving/ECHO_modules/blob/watershed-geo/utilities.py)
data_set_widget = show_data_set_widget( data_sets ) 

#Note: we might consider limiting to CWA/SDWA given the watershed-focus of this notebook

Get the data from the Stony Brook University database. 

In [ ]:
# Use the selection from the widget above to save data from that program
program = data_sets[ data_set_widget.value ]
if ( use_zips ):
    program_results = program.store_results( region_type="Zip Code", region_value=zips_selected )
else:
    program_results = program.store_results( region_type="Watershed", region_value=hucs, state=states_w.value )
program_data = None

# Print the data about the program
if ( program_results is not None and program_results.dataframe is not None ):
    program_data = program_results.dataframe.copy()

    display( program_data )
else:
    print( "There is no data for this data set in this watershed.")

Pollutant discharge data. We will pull up what facilities reported discharging into this watershed. 

In [ ]:
dmr_data_set = data_sets['2020 Discharge Monitoring']
dmr_results = dmr_data_set.store_results( region_type="Watershed", region_value=hucs, state=states_w.value )
dmr_data = None
if ( dmr_results is not None ):
    dmr_data = dmr_results.dataframe.copy()

    display( dmr_data )
else:
    print( "There is no discharge monitoring data for this data set in this watershed.")

### At this point you can follow two different approaches.  

1.  You can choose to see a set of facilities in the watersheds, then view and select from the pollutants discharged by those facilities. Run the cells starting with A below.
2.  You can choose to see all of the pollutants from all of the facilities in the selected watersheds, then pick specific pollutants and see which of the facilities are discharge. Skip down to the cells starting at B.

Note:  Run through the cells in each section in sequence.  For example, if you run through section A, then section B, then want to come back and look at different facilities/pollutants in section A, be sure and run the cells in section A again in the order they are presented.

### A.  Select one or more of the facilities.  To select multiple, use Ctrl+click to add, Shift+click to extend.

In [ ]:
# Create a widget that lists the facilities with discharge monitoring in the
#   selected watershed area(s)
facility_list = dmr_data[ 'FAC_NAME' ].unique()
facility_list.sort()
facility_w = widgets.SelectMultiple(
    options= facility_list,
    description='Facility:',
    disabled=False,
)
# Display the widget
facility_w

Select one or more of the pollutants from these facilities.

In [ ]:
# Create a widget for selection of pollutant(s) of interest
selected_facs = facility_w.value
dmr_fac_data = dmr_data[ dmr_data['FAC_NAME'].isin( selected_facs )]

param_list = dmr_fac_data[ 'PARAMETER_DESC' ].unique()
param_list.sort()
param_w = widgets.SelectMultiple(
    options = param_list,
    description = 'Parameter:',
    disabled = False
)
# Display the widget
param_w

These are the discharges of the selected parameters from the selected facilities.

In [ ]:
# Fetch the data associated with the selected facility(ies) and pollutant(s)
selected_params = param_w.value
dmr_param_data = dmr_fac_data[ dmr_fac_data['PARAMETER_DESC'].isin( selected_params )]
# print( dmr_param_data )
print( dmr_param_data.loc[:,[ 'FAC_NAME', 'PARAMETER_CODE', 'PARAMETER_DESC', 'LIMIT_VALUE_NMBR', 'LIMIT_VALUE_QUALIFIER_CODE',
              'LIMIT_VALUE_TYPE_CODE', 'VIOLATION_CODE']] )

View a map of only those selected facilities that have discharged the selected pollutant.

In [ ]:
# Map facilities matching criteria using Folium
if ( dmr_param_data is not None ):
    map_of_facilities = mapper(dmr_param_data) # Some errors not caught here...
    this_watershed = gdf.loc[gdf.index.isin( watershed_w.value )]
    w = folium.GeoJson(
      this_watershed,
      name = "Watershed",
    ).add_to(map_of_facilities)
    display( map_of_facilities ) 
else:
    print( "There are no facilities in the watershed." ) 

Write the data to a file.

In [ ]:
dmr_param_data.to_csv( 'Discharge-by-facilities-pollutants.csv')

### B. We start again with all of the facilities in the selected watersheds.  We view the pollutants discharged by these facilities

Select specific pollutants.

In [ ]:
param_list = dmr_data[ 'PARAMETER_DESC' ].unique()
param_list.sort()
param_w = widgets.SelectMultiple(
    options = param_list,
    description = 'Parameter:',
    disabled = False
)
param_w

Map the facilities that discharge these pollutants.

In [ ]:
selected_params = param_w.value
dmr_param_data = dmr_data[ dmr_data['PARAMETER_DESC'].isin( selected_params )]
dmr_param_fac_data = dmr_param_data[['FAC_NAME','DFR_URL','FAC_LAT','FAC_LONG']].drop_duplicates()

if ( dmr_param_data is not None ):
    map_of_facilities = mapper(dmr_param_fac_data) # Some errors not caught here...
    this_watershed = gdf.loc[gdf.index.isin( watershed_w.value )]
    w = folium.GeoJson(
      this_watershed,
      name = "Watershed",
    ).add_to(map_of_facilities)
    display( map_of_facilities ) 
else:
    print( "There are no facilities in the watershed." ) 

Select from among the facilities discharging these pollutants.  For example, you might select facilities lying along a particular waterway within the watersheds.

In [ ]:
facility_list = dmr_param_fac_data[ 'FAC_NAME' ].tolist()
facility_list.sort()
facility_w = widgets.SelectMultiple(
    options= facility_list,
    description='Facility:',
    disabled=False,
)
facility_w

View the data for the pollutants discharged by these facilities.

In [ ]:
selected_facs = facility_w.value
dmr_fac_data = dmr_param_data[ dmr_param_data['FAC_NAME'].isin( selected_facs )]
dmr_fac_data

Write the data to a file.

In [ ]:
dmr_fac_data.to_csv( 'Discharge-by-pollutants-facilitys.csv')

### C. This section ... some experimental stuff

In [ ]:
pollutants_w = widgets.Dropdown(
    options= sorted(list(dmr_data["PARAMETER_DESC"].unique())),
    description='Pollutant:',
    disabled=False,
)
pollutants_w

In [ ]:
dmr_data.columns.values.tolist()

In [ ]:
this_dmr_data = dmr_data.loc[dmr_data["PARAMETER_DESC"] == pollutants_w.value]

# Cataloguing missing information
print((this_dmr_data["DMR_VALUE_NMBR"].isna().sum() / len(this_dmr_data) ) *100) # Percent of DMR values for this pollutant missing
print(100* this_dmr_data.drop_duplicates(subset=['LIMIT_VALUE_ID'])["LIMIT_VALUE_NMBR"].isna().sum()/len(this_dmr_data.drop_duplicates(subset=['LIMIT_VALUE_ID']))) # percent of LIMIT_VALUE_NMBR not reported. Does not account for stays.
#NMBR_OF_SUBMISSION - The attribute stores the number of months for submitting the DMRs for the limit set (e.g., monthly = 1, semi-annually = 6, quarterly = 3); this data element will be blank for Unscheduled Limit Sets. Must be greater than or equal to NMBR_OF_REPORT and be divisible by NMBR_OF_REPORT.
#NMBR_OF_REPORT - The number of months in the monitoring period covered by the DMR (e.g., monthly = 1, quarterly = 3, semi-annually = 6). 
#For example, if NMBR_OF_REPORT is 3, there should be 4 quarterly reports here. If it is 6, there should be 2.
# ??? NMBR_OF_REPORT not in the columns?
# display(this_dmr_data.groupby(["LIMIT_VALUE_ID"])[['NMBR_OF_REPORT']].agg({"count", "first"})) #first is just a cheap way to record the actual NMBR_OF_REPORT value

# Cataloguing important numbers
print((len(this_dmr_data) / len(dmr_data)) *100) # share of all reports accounted for by this pollutant
print(str(this_dmr_data.drop_duplicates(subset=['LIMIT_VALUE_ID'])["LIMIT_VALUE_NMBR"].sum())+" "+this_dmr_data["LIMIT_UNIT_DESC"].unique()[0]) # total permitted value LIMIT_VALUE_NMBR. Assumes units are same.
print(np.nanmedian(this_dmr_data["EXCEEDENCE_PCT"])) #median percent over permitted value for this pollutant, excluding NaNs. If the output is nan, all exceedance values are NaN

In [ ]:
grouped = dmr_fac_data.loc[:,[ 'FAC_NAME', 'PARAMETER_CODE', 'PARAMETER_DESC', 'LIMIT_VALUE_NMBR', 'LIMIT_VALUE_QUALIFIER_CODE',
              'LIMIT_VALUE_TYPE_CODE', 'VIOLATION_CODE']].groupby( ['FAC_NAME', 'PARAMETER_CODE'] )
for name, group in grouped:
    print( name )
    # group.to_csv( "{}.csv".format( name ))
display( grouped )


To export the above data as a spreadsheet you can view in Excel, run the code block below. The fiel will appear in the "Files" tab on Google Colab (click the folder on the left hand side of the screen.

In [ ]:
dmr_data.to_csv("dmr_data.csv")